# SNLP project - recipe summarisation

## Abstract

**Objective:**

Train and compare models which summarises recipes given the ingredients, instructions, and other details. 
The data set is sourced from the Spoonacular API. 

**Candidate summarisation strategies**

We will consider three summarisation strategies.
1. summarisation by producing a text summary of the recipes
2. summarisation by producing a list of classes to which the recipes belong (e.g. vegan, oven-free) 
3. summarisation by predicting numeric characteristic (e.g. preparation time)

## Data cleaning

In [253]:
import pandas as pd
import numpy as np

In [259]:
df = pd.read_csv("./data/cleaneddata.csv", index_col=0)

In [260]:
df

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,weightWatcherSmartPoints,healthScore,pricePerServing,extendedIngredients,id,title,readyInMinutes,servings,summary,instructions,analyzedInstructions
0,False,False,True,False,False,False,False,False,13,24.0,274.60,"[{'id': 1032028, 'aisle': 'Ethnic Foods;Spices...",780001,Pesto Chicken Zoodles,45,3,Need a <b>gluten free and primal main course</...,"To get started, place 1/4 cup of olive oil int...","[{'name': '', 'steps': [{'number': 1, 'step': ..."
1,True,True,True,True,True,False,False,False,8,100.0,126.27,"[{'id': 4053, 'aisle': 'Oil, Vinegar, Salad Dr...",663559,Tomato and lentil soup,45,4,Tomato and lentil soup takes about <b>about 45...,Saut onion and garlic in olive oil for 5 minut...,"[{'name': '', 'steps': [{'number': 1, 'step': ..."
2,True,True,True,True,False,False,False,False,3,40.0,43.68,"[{'id': 1032035, 'aisle': 'Spices and Seasonin...",716195,Spicy Indian-Style Hummus,45,12,Spicy Indian-Style Hummus might be just the ho...,<p>Rinse the chickpeas and soak for 8 hours or...,"[{'name': '', 'steps': [{'number': 1, 'step': ..."
3,True,False,True,True,False,False,False,False,4,4.0,86.32,"[{'id': 1123, 'aisle': 'Milk, Eggs, Other Dair...",662276,Sun Dried Tomato and Herb Baked Eggs,25,1,Sun Dried Tomato and Herb Baked Eggs might be ...,<ol><li>Preheat oven to 350 F.</li><li>Cover t...,"[{'name': '', 'steps': [{'number': 1, 'step': ..."
4,True,False,False,False,False,False,False,False,21,3.0,59.75,"[{'id': 1001, 'aisle': 'Milk, Eggs, Other Dair...",633970,Banana & Oreo Muffin,45,4,Banana & Oreo Muffin is a <b>lacto ovo vegetar...,"Sift the flour, baking power and baking soda t...","[{'name': '', 'steps': [{'number': 1, 'step': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,True,False,False,False,False,False,False,False,21,1.0,111.56,"[{'id': 1001, 'aisle': 'Milk, Eggs, Other Dair...",167600,Citrus Cheesecake with Berries,845,16,"For <b>$1.12 per serving</b>, this recipe <b>c...",1 Move oven rack...,"[{'name': '', 'steps': [{'number': 1, 'step': ..."
52,False,False,True,False,False,False,False,False,13,3.0,118.41,"[{'id': 1002050, 'aisle': 'Baking', 'image': '...",124589,Peach Tart with Island Crust,135,8,Watching your figure? This gluten free recipe ...,NaN,NaN
53,True,True,True,True,False,False,False,False,4,25.0,176.93,"[{'id': 2048, 'aisle': 'Oil, Vinegar, Salad Dr...",757565,Shredded Beet Bowl with Nori Confetti,45,6,"For <b>$1.54 per serving</b>, this recipe <b>c...",1. Finely grate or shred beets with food proce...,"[{'name': '', 'steps': [{'number': 1, 'step': ..."
54,False,False,True,True,False,False,False,False,3,8.0,250.63,"[{'id': 1002014, 'aisle': 'Spices and Seasonin...",5569,Grilled Lime Shrimp (from Recipezaar),22,4,Grilled Lime Shrimp (from Recipezaar) might be...,NaN,NaN


## Code

#### 1. Summarisation by producing a text summary of the recipes

#### 2. Summarisation by producing a list of classes to which the recipes belong (e.g. vegan, oven-free)

#### 3. Summarisation by predicting numeric characteristic (e.g. preparation time)